In [ ]:
import asyncio
from typing import AsyncGenerator

async def echo_generator() -> AsyncGenerator[str, str]:
    """
    An async generator that:
      1. Yields a “ready” message when first started.
      2. Then for each value sent in, yields back “echo: {value}”.
    """
    # First yield – no value comes in yet
    received = yield "ready"
    # Now loop forever, echoing back whatever we receive next
    while True:
        received = yield f"echo: {received}"

async def main():
    # Create the generator
    agen = echo_generator()

    # 1) “Prime” the generator: must send None to get the first yield
    first = await agen.asend(None)
    print(first)  # → "ready"

    # 2) Send in “hello” and await the next yield
    resp1 = await agen.asend("hello")
    print(resp1)  # → "echo: hello"

    # 3) Send in “world” and await the next yield
    resp2 = await agen.asend("world")
    print(resp2)  # → "echo: world"

    # 4) Close the generator
    try:
        await agen.asend(None)
    except StopAsyncIteration:
        print("Generator has exited.")

# Run it
await main()


# Generator Functions

In [ ]:
def myfunc():
    """
    A simple function that does nothing.
    """
    return 1
    return 2
    return 3

# This function is not used in the async context, but is included to match the original request.

In [ ]:
def myfunc():
    """
    A simple function that does nothing.
    """
    yield 1
    yield 2
    yield 3

# This function is not used in the async context, but is included to match the original request.

In [ ]:
myfunc()

In [ ]:
g = myfunc()

In [ ]:
next(g)

In [ ]:
g.gi_frame.f_lineno

In [ ]:
# we can create, using this, our own baby verion of asyncio

# we'll create a list
# we'll put those generators in the list
# we'll iterate over the list, giving each generator a chance to run
# when a generator is done (it raises StopIteration), we remove it from the list
# this is a very simple event loop

In [ ]:
def mygen(id_number, maxnum):
    """
    A simple generator that yields numbers up to a maximum.
    """
    for i in range(maxnum):
        yield f"Generator {id_number}: {i}"

In [ ]:
g1 = mygen(1, 5)
g2 = mygen(2, 7)
g3 = mygen(3, 3)

In [ ]:
generators = [g1, g2, g3]

In [ ]:
while generators:
    for one_g in generators:
        try:
            print(next(one_g))
        except StopIteration:
            # If the generator is done, remove it from the list
            generators.remove(one_g)
print("All generators are done.")

In [ ]:
# Mapping this to an asyncio-like event loop:

# - In 'asyncio' , we don't define regular functions. Rather, we define coroutines using 'async def'.
# Just as generators functions, when we run them,  gives us generators objects, async def functions return *coroutines*. 
# You don't run a corouting directly. Rather, you put it on the event loop, which will run it for you.
# Instead of 'yield' in generators, in 'asyncio' we use a term called 'await'. This means two things are once: First, that
# we're are waiting for a value from something that might take a while. Second: While we're waiting, we are willing to goto sleep, 
# a la yield, and let other things run. So, 'await' is like 'yield' in that sense.

In [ ]:
async def hello():
    print(("Hello, world!"))

In [ ]:
type(hello)

In [ ]:
import dis
dis.show_code(hello)

In [ ]:
hello()

In [ ]:
# WE don't run coroutines directly. We put them on the event loop, which will run them for us.

In [ ]:
await hello()

In [ ]:
import asyncio

async def hello():
    await asyncio.sleep(1)
    print(("Hello, world!"))

In [ ]:
await hello()

In [ ]:
import random
async def hello(number):
    await asyncio.sleep(random.randint(0, 4))
    print((f"Hello, world: {number}!"))
    return number

In [ ]:
tasks = [asyncio.create_task(hello(i)) for i in range(5)]
result = await asyncio.gather(*tasks)



In [ ]:
result

In [ ]:
async def up(maximum):
    for i in range(maximum):
        await asyncio.sleep(1)
        print(f"Up: {i}")
        await asyncio.sleep(0)       # <-- explicit yield back to the loop

async def down(maximum):
    for i in range(maximum, 0, -1):
        await asyncio.sleep(1)
        print(f"Down: {i}")

async def powers(n):
    for i in range(2, 8):
        await asyncio.sleep(1)
        print(f"Powers: {n ** i}")

async def run_tasks():
    coros = [up(4), down(4), powers(4)] 
    async with asyncio.TaskGroup() as tg:
        for coro in coros:
            tg.create_task(coro)


    result = await coros[0] # RuntimeError: cannot reuse already awaited coroutine

# Once a coroutine has been awaited and finished, it cannot be awaited again.
# This is because a coroutine is an awaitable object that runs only once—after it completes, its state is "exhausted".
# Trying to await it again will raise a RuntimeError.
# If you want to run the same logic multiple times, you must create a new coroutine object each time.

results = await run_tasks() 


Under the hood, a Python `async def` function isn’t a fresh piece of code every time you `await` it—it’s more like a stateful generator:

1. **Coroutine objects are single‐use generators**
   When you call an `async def` function, e.g. `coro = hello(…)`, Python builds a *coroutine object*, which internally is a generator‐style state machine. You then “drive” it to completion by awaiting it (directly or via `gather`, etc.).

2. **Once it runs to completion, its state is done**
   Like a normal generator, when it returns (i.e. reaches the end or hits a `return`), it raises `StopIteration` internally. At that point its internal instruction pointer is at the end, local variables have been torn down, and there’s no “rewind” mechanism.

3. **Re‐awaiting a finished coroutine is invalid**
   If you try:

   ```python
   coro = hello(1)
   await coro        # first time: works
   await coro        # second time: RuntimeError
   ```

   you’ll get:

   ```
   RuntimeError: cannot reuse already awaited coroutine
   ```

   That error exists because Python won’t implicitly recreate or rewind the coroutine object for you.

4. **To run the same logic again, call the function again**
   You need a *new* coroutine object each time:

   ```python
   await hello(1)            # runs once
   await hello(1)            # brand‐new coroutine, runs again
   ```

   Or, if you stored it, recreate:

   ```python
   coro = hello(1)
   await coro
   # …later…
   coro = hello(1)           # NEW object
   await coro
   ```

---

#### Why design it this way?

* **Statefulness:** Coroutines can suspend at `await`, keep internal locals, stack frames, exception handlers, etc. They’re inherently stateful.
* **Performance & simplicity:** Treating them as one‐shot generators keeps the implementation simple and avoids hidden memory resets or surprising side effects.
* **Explicit is better than implicit:** If you really wanted to reuse logic, you explicitly call the function again, making it clear you’re spinning up a fresh state machine.

In short: a finished coroutine is “exhausted” just like a generator after it hits `return`, and you must call the `async def` function again to get a fresh coroutine to await.


# How to Trap Exceptions in a TaskGroup

## Catch the `ExceptionGroup` when the group exits

By design, if *any* child task raises, the TaskGroup:

1. Cancels all *other* children
2. Propagates an **`ExceptionGroup`** out of the `async with` block

You can catch that exception and unpack it with the new `except*` syntax (Python 3.11+), or inspect its `.exceptions` list:

```python
import asyncio

async def bad(i):
    await asyncio.sleep(i * 0.1)
    raise RuntimeError(f"task {i} failed")

async def main():
    try:
        async with asyncio.TaskGroup() as tg:
            # schedule a few tasks, one of which will error
            tg.create_task(bad(1))
            tg.create_task(bad(2))
            tg.create_task(bad(3))
    # catch *only* RuntimeErrors from the group
    except* RuntimeError as group:
        for exc in group.exceptions:
            print("Caught child error:", exc)
    # (you could also do `except* Exception as group:` to catch all)
    else:
        print("all tasks succeeded")

asyncio.run(main())
```

**What’s happening?**

* As soon as one `bad(i)` raises, the TaskGroup tears down: all remaining tasks are cancelled, and an `ExceptionGroup` is thrown.
* `except* RuntimeError as group:` pulls out just the `RuntimeError` instances into `group.exceptions`.

